In [44]:
from torchvision import datasets
from torchvision.transforms import ToTensor

In [45]:
train_data = datasets.MNIST(
    root = 'data',
    train = True,
    transform = ToTensor(),
    download = True
)
test_data = datasets.MNIST(
    root = 'data',
    train = False,
    transform = ToTensor(),
    download = True
)

In [46]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [47]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [48]:
train_data.data.shape

torch.Size([60000, 28, 28])

In [49]:
train_data.targets.size()

torch.Size([60000])

In [50]:
from torch.utils.data import DataLoader

loaders = {

    'train': DataLoader(train_data,
                        batch_size=100,
                        shuffle=True,
                        num_workers=1),
    
    'test': DataLoader(test_data,
                        batch_size=100,
                        shuffle=True,
                        num_workers=1)

}

In [51]:
loaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x34e35f590>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x34e35f920>}

In [52]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class CNN(nn.Module):

    def __init__(self):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
    
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        self = self.fc2(x)

        return F.softmax(x)

In [64]:
import torch
print(torch.backends.mps.is_available())

# device = torch.device('mps' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

model = CNN().to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)

loss_fn = nn.CrossEntropyLoss()

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(loaders['train']):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 20 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(loaders["train"].dataset)} ({100. * batch_idx/len(loaders["train"]):.0f}%)]\t{loss.item():.6f}')

def test():
    model.eval()

    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in loaders['test']:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_fn(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(loaders['test'].dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy {correct}/{len(loaders["test"].dataset)} ({100. * correct / len(loaders["test"].dataset):.0f}%\n)')
    

True


In [65]:
import time

#device is already set up to CPU

start_time_cpu = time.time()

for epoch in range(1,11):
    train(epoch)
    test()

end_time_cpu = time.time() 
total_time_cpu = end_time_cpu - start_time_cpu
#total_time_cpu


/var/folders/8_/qqhcw4rd4m72212dbpl453gh0000gn/T/ipykernel_47958/845714647.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)


Train Epoch: 1 [0/60000 (0%)]	3.907997
Train Epoch: 1 [1280/60000 (2%)]	3.849427
Train Epoch: 1 [2560/60000 (4%)]	3.786571
Train Epoch: 1 [3840/60000 (6%)]	3.678655
Train Epoch: 1 [5120/60000 (9%)]	3.704136
Train Epoch: 1 [6400/60000 (11%)]	3.652076
Train Epoch: 1 [7680/60000 (13%)]	3.676493
Train Epoch: 1 [8960/60000 (15%)]	3.650297
Train Epoch: 1 [10240/60000 (17%)]	3.663260
Train Epoch: 1 [11520/60000 (19%)]	3.515284
Train Epoch: 1 [12800/60000 (21%)]	3.679663
Train Epoch: 1 [14080/60000 (23%)]	3.538126
Train Epoch: 1 [15360/60000 (26%)]	3.574050
Train Epoch: 1 [16640/60000 (28%)]	3.626576
Train Epoch: 1 [17920/60000 (30%)]	3.578941
Train Epoch: 1 [19200/60000 (32%)]	3.732123
Train Epoch: 1 [20480/60000 (34%)]	3.569031
Train Epoch: 1 [21760/60000 (36%)]	3.533845
Train Epoch: 1 [23040/60000 (38%)]	3.675719
Train Epoch: 1 [24320/60000 (41%)]	3.687897
Train Epoch: 1 [25600/60000 (43%)]	3.716060
Train Epoch: 1 [26880/60000 (45%)]	3.745520
Train Epoch: 1 [28160/60000 (47%)]	3.644723
Trai

In [66]:
device = torch.device('mps') #switching device to GPU

model = CNN().to(device)

start_time_gpu = time.time()

for epoch in range(1,11):
    train(epoch)
    test()

end_time_gpu = time.time() 
total_time_gpu = end_time_gpu - start_time_gpu
#total_time_gpu


/var/folders/8_/qqhcw4rd4m72212dbpl453gh0000gn/T/ipykernel_47958/845714647.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)


Train Epoch: 1 [0/60000 (0%)]	3.912474
Train Epoch: 1 [1280/60000 (2%)]	3.909939
Train Epoch: 1 [2560/60000 (4%)]	3.911856
Train Epoch: 1 [3840/60000 (6%)]	3.912793
Train Epoch: 1 [5120/60000 (9%)]	3.913304
Train Epoch: 1 [6400/60000 (11%)]	3.911704
Train Epoch: 1 [7680/60000 (13%)]	3.913788
Train Epoch: 1 [8960/60000 (15%)]	3.912409
Train Epoch: 1 [10240/60000 (17%)]	3.912948
Train Epoch: 1 [11520/60000 (19%)]	3.911966
Train Epoch: 1 [12800/60000 (21%)]	3.912636
Train Epoch: 1 [14080/60000 (23%)]	3.912676
Train Epoch: 1 [15360/60000 (26%)]	3.913051
Train Epoch: 1 [16640/60000 (28%)]	3.911372
Train Epoch: 1 [17920/60000 (30%)]	3.912899
Train Epoch: 1 [19200/60000 (32%)]	3.912570
Train Epoch: 1 [20480/60000 (34%)]	3.913175
Train Epoch: 1 [21760/60000 (36%)]	3.912596
Train Epoch: 1 [23040/60000 (38%)]	3.913628
Train Epoch: 1 [24320/60000 (41%)]	3.912153
Train Epoch: 1 [25600/60000 (43%)]	3.911696
Train Epoch: 1 [26880/60000 (45%)]	3.912830
Train Epoch: 1 [28160/60000 (47%)]	3.912663
Trai

KeyboardInterrupt: 

In [ ]:


# print(f"Training on {device_name} took: {total_time:.2f} seconds")
# return total_time
# time_difference = cpu_time - gpu_time
# print(f"Time difference (CPU - GPU): {time_difference:.2f} seconds")
